# 2022 League Two EDA

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [13]:
team_stats = pd.read_csv('../../footy_data/league_two_teams_2022.csv')
games = pd.read_csv('../../footy_data/league_two_games_2022.csv')

In [14]:
games = games.loc[games['status'] == 'complete']
games
#Played matches

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Exeter City,Bradford City,Samuel Allison,1,0.00,0.00,...,2.55,3.10,2.85,1.43,2.15,3.65,7.00,1.95,1.83,St James Park (Exeter- Devon)
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Forest Green Rovers,Sutton United,James Bell,1,0.00,0.00,...,1.83,3.50,4.20,1.35,1.95,3.30,6.50,1.91,1.87,The Fully Charged New Lawn (Nailsworth- Glouce...
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Harrogate Town,Rochdale,Anthony Coggins,1,0.00,0.00,...,2.20,3.25,3.25,1.37,2.05,3.35,6.50,1.87,1.91,EnviroVent Stadium (Harrogate- North Yorkshire)
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Hartlepool United,Crawley Town,Scott Oldham,1,0.00,0.00,...,2.20,3.20,3.30,1.39,2.10,3.45,6.75,1.91,1.87,Victoria Park (Hartlepool- County Durham)
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Mansfield Town,Bristol Rovers,Joshua Smith,1,0.00,0.00,...,2.35,3.15,3.05,1.43,2.15,3.60,7.00,1.95,1.83,One Call Stadium (Mansfield- Nottinghamshire)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,1632578400,Sep 25 2021 - 2:00pm,complete,NaN,Salford City,Northampton Town,Anthony Backhouse,9,1.60,1.75,...,2.00,3.35,3.65,1.42,2.15,3.70,7.25,2.00,1.77,The Peninsula Stadium (Salford- Greater Manche...
103,1632578400,Sep 25 2021 - 2:00pm,complete,NaN,Scunthorpe United,Port Vale,Rob Lewis,9,1.00,1.75,...,3.80,3.40,1.95,1.43,2.25,3.90,7.75,2.10,1.71,Sands Venue Stadium (Scunthorpe)
104,1632578400,Sep 25 2021 - 2:00pm,complete,NaN,Sutton United,Carlisle United,Simon Mather,9,2.00,1.00,...,2.35,3.25,3.00,1.40,2.10,3.55,6.75,1.91,1.87,VBS Community Stadium (Sutton- Surrey)
105,1632578400,Sep 25 2021 - 2:00pm,complete,NaN,Swindon Town,Colchester United,Andrew Kitchen,9,1.00,2.00,...,2.15,3.40,3.25,1.35,1.95,3.25,6.00,1.83,1.95,The County Ground (Swindon- Wiltshire)


In [32]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)


In [45]:
xg_view = games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]
xg_view[50:110]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
50,Hartlepool United,Carlisle United,1.35,0.87,2,1
51,Leyton Orient,Bradford City,1.81,0.91,2,0
52,Salford City,Newport County,1.70,1.73,3,0
53,Scunthorpe United,Tranmere Rovers,1.41,0.89,1,0
54,Sutton United,Oldham Athletic,1.36,1.43,1,2
55,Swindon Town,Mansfield Town,1.24,1.78,1,0
56,Walsall,Stevenage,0.99,1.10,1,0
57,Bradford City,Walsall,2.05,1.02,1,1
58,Bristol Rovers,Crawley Town,1.04,2.23,1,0
59,Carlisle United,Salford City,1.73,1.52,2,1


# Goal Analysis

In [34]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100

In [35]:
print(f'There were {season_goal_total} goals scored this season')

There were 241 goals scored this season


In [36]:
print(f'There were {season_1h_goal_total} goals scored in the 1H this season, ')

There were 118 goals scored in the 1H this season, 


In [37]:
round(season_1h_goal_total/season_goal_total, 2)
#% of first half goals

0.49

In [38]:
games['total_goals_at_half_time'].count()

107

In [39]:
games['total_goals_at_half_time'].value_counts()

0    42
1    29
2    23
3     9
4     4
Name: total_goals_at_half_time, dtype: int64

# Adding more columns

In [40]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

Let's write a function to help fill xg in our games df

In [41]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]


In [42]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [43]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [26]:
fill_stats(team_stats['common_name'][0])
fill_stats(team_stats['common_name'][1])
fill_stats(team_stats['common_name'][2]) 
fill_stats(team_stats['common_name'][3]) 
fill_stats(team_stats['common_name'][4]) 
fill_stats(team_stats['common_name'][5]) 
fill_stats(team_stats['common_name'][6]) 
fill_stats(team_stats['common_name'][7]) 
fill_stats(team_stats['common_name'][8]) 
fill_stats(team_stats['common_name'][9]) 
fill_stats(team_stats['common_name'][10]) 
fill_stats(team_stats['common_name'][11]) 
fill_stats(team_stats['common_name'][12]) 
fill_stats(team_stats['common_name'][13]) 
fill_stats(team_stats['common_name'][14]) 
fill_stats(team_stats['common_name'][15]) 
fill_stats(team_stats['common_name'][16]) 
fill_stats(team_stats['common_name'][17]) 
fill_stats(team_stats['common_name'][18]) 
fill_stats(team_stats['common_name'][19]) 
fill_stats(team_stats['common_name'][20]) 
fill_stats(team_stats['common_name'][21]) 
fill_stats(team_stats['common_name'][22]) 
fill_stats(team_stats['common_name'][23]) 

#Reading in stats

In [27]:
games

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,home_xg,away_xg,home_xg_against,away_xg_against,home_total_corners,away_total_corners
0,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Exeter City,Bradford City,Samuel Allison,1,0.00,0.00,...,7.00,1.95,1.83,St James Park (Exeter- Devon),1.29,1.63,1.33,1.28,4.11,6
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Forest Green Rovers,Sutton United,James Bell,1,0.00,0.00,...,6.50,1.91,1.87,The Fully Charged New Lawn (Nailsworth- Glouce...,1.62,1.44,1.1,1.25,5.67,6.13
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Harrogate Town,Rochdale,Anthony Coggins,1,0.00,0.00,...,6.50,1.87,1.91,EnviroVent Stadium (Harrogate- North Yorkshire),1.39,1.29,1.13,1.21,6.78,5.33
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Hartlepool United,Crawley Town,Scott Oldham,1,0.00,0.00,...,6.75,1.91,1.87,Victoria Park (Hartlepool- County Durham),0.96,1.03,0.95,1.28,4.11,3.78
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Mansfield Town,Bristol Rovers,Joshua Smith,1,0.00,0.00,...,7.00,1.95,1.83,One Call Stadium (Mansfield- Nottinghamshire),1.38,0.86,1.19,1.25,4.67,4.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,1632578400,Sep 25 2021 - 2:00pm,complete,NaN,Salford City,Northampton Town,Anthony Backhouse,9,1.60,1.75,...,7.25,2.00,1.77,The Peninsula Stadium (Salford- Greater Manche...,1.38,1.2,1.25,1.1,4.56,4.44
103,1632578400,Sep 25 2021 - 2:00pm,complete,NaN,Scunthorpe United,Port Vale,Rob Lewis,9,1.00,1.75,...,7.75,2.10,1.71,Sands Venue Stadium (Scunthorpe),0.78,1.22,1.41,1.12,4.56,6.11
104,1632578400,Sep 25 2021 - 2:00pm,complete,NaN,Sutton United,Carlisle United,Simon Mather,9,2.00,1.00,...,6.75,1.91,1.87,VBS Community Stadium (Sutton- Surrey),1.44,1.3,1.25,1.06,6.13,4.44
105,1632578400,Sep 25 2021 - 2:00pm,complete,NaN,Swindon Town,Colchester United,Andrew Kitchen,9,1.00,2.00,...,6.00,1.83,1.95,The County Ground (Swindon- Wiltshire),1.32,1.02,1.31,1.51,3.33,5.25


# Some more EDA

Let's write a function to help fill xg in our games df

In [28]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go



xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats.head()

xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

fill_stats('Fulham')
fill_stats('Blackburn Rovers')
fill_stats('West Bromwich Albion')
fill_stats('Queens Park Rangers')
fill_stats('AFC Bournemouth')
fill_stats('Middlesbrough')
fill_stats('Barnsley')
fill_stats('Millwall')
fill_stats('Sheffield United')
fill_stats('Reading')
fill_stats('Cardiff City')
fill_stats('Nottingham Forest')
fill_stats('Hull City')
fill_stats('Blackpool')
fill_stats('Luton Town')
fill_stats('Stoke City')
fill_stats('Swansea City')
fill_stats('Derby County')
fill_stats('Huddersfield Town')
fill_stats('Preston North End')
fill_stats('Coventry City')
fill_stats('Peterborough United')
fill_stats('Birmingham City')
fill_stats('Bristol City')

#Reading in stats

# Some more EDA

KeyError: 'Fulham'

# Over/Under 2.5?

In [ ]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [29]:
games['over/under'].value_counts()

KeyError: 'over/under'

# xG Over/Under 2.5? 

In [30]:
games['xg_over/under'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 2.99, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['xg_over/under_1.99'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 1.99, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [31]:
games['xg_over/under'].value_counts()

0    59
1    48
Name: xg_over/under, dtype: int64

In [ ]:
games['xg_over/under_1.99'].value_counts()

# First Week

In [ ]:
games['over/under_1.5'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 1.5, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [ ]:
first_matchweek = games[0:12]

In [ ]:
first_matchweek['over/under'].value_counts()

In [ ]:
first_matchweek['over/under_1.5'].value_counts()

In [ ]:
first_matchweek['xg_over/under'].value_counts()

In [ ]:
first_matchweek['xg_over/under_1.99'].value_counts()

In [ ]:
games['home_team_goal_count_half_time'].sum()

In [ ]:
games['away_team_goal_count_half_time'].sum()

# Let's take out the final week now and check percentage

In [ ]:
first_41 = games[0:540]
first_41['over/under'].value_counts()

In [ ]:
217/540
# Percentage of games that went over

# When were these goals scored?

Second half stoppage time goals will start at 91 i.e. 90+1 will be 91
First half stoppage time goals will start at 100 i.e. 45+1 = 110 ----

Change stoppage time goals so we can cast them as floats

In [ ]:
goal_timings = goal_timings.replace(to_replace="90'1", value = "91")
goal_timings = goal_timings.replace(to_replace="90'2", value = "92")
goal_timings = goal_timings.replace(to_replace="90'3", value = "93")
goal_timings = goal_timings.replace(to_replace="90'4", value = "94")
goal_timings = goal_timings.replace(to_replace="90'5", value = "95")
goal_timings = goal_timings.replace(to_replace="90'6", value = "96")
goal_timings = goal_timings.replace(to_replace="90'7", value = "97")
goal_timings = goal_timings.replace(to_replace="90'8", value = "98")
goal_timings = goal_timings.replace(to_replace="90'9", value = "99")
goal_timings = goal_timings.replace(to_replace="90'10", value = "100")
goal_timings = goal_timings.replace(to_replace="90'11", value = "101")
goal_timings = goal_timings.replace(to_replace="90'12", value = "102")
goal_timings = goal_timings.replace(to_replace="45'1", value = "110")
goal_timings = goal_timings.replace(to_replace="45'2", value = "111")
goal_timings = goal_timings.replace(to_replace="45'3", value = "112")
goal_timings = goal_timings.replace(to_replace="45'4", value = "113")

In [ ]:
goal_timings['All_goal_timings'] = goal_timings['All_goal_timings'].astype('int')

In [ ]:
timing_totals = goal_timings['All_goal_timings'].value_counts()
timing_totals = pd.DataFrame(timing_totals)

In [ ]:
timing_totals = timing_totals.rename_axis("minute", axis="columns")
timing_totals = timing_totals.rename(columns={"All_goal_timings":"count"})

In [ ]:
timing_totals['section'] = ""
timing_totals.head()

# In how many games did a team have more xG but lose?